
# Limpieza de los datasets


In [179]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import Normalizer, OneHotEncoder

pd.set_option('display.max_columns', None)

## 1.- Campus selection

In [180]:
ruta = 'data/raw_data/Campus_selection/Campus_Selection.csv'
df = pd.read_csv(ruta)
df.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed


In [181]:
df['status'].value_counts()

status
Placed        148
Not Placed     67
Name: count, dtype: int64

In [182]:
minoritaria = df['status'].value_counts().min()
mayoritaria = df['status'].value_counts().max()
IR = np.round(mayoritaria/minoritaria, 4)
print(f'IR: {IR}')

IR: 2.209


In [183]:
X = df.iloc[:, :-1]
y = df['status']

In [184]:
y.head()

0        Placed
1        Placed
2        Placed
3    Not Placed
4        Placed
Name: status, dtype: object

In [185]:
y = y.replace({'Placed':0, 'Not Placed':1})
y.head()

/tmp/ipykernel_4241/942715215.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'Placed':0, 'Not Placed':1})


0    0
1    0
2    0
3    1
4    0
Name: status, dtype: int64

### Normalizar numericos

In [186]:
numericos = X.select_dtypes(include='number')
normalizer = Normalizer()
numericos_norm = normalizer.fit_transform(numericos)
numericos_norm = pd.DataFrame(numericos_norm, columns = numericos.columns)
numericos_norm.head()

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p
0,0.006649,0.445506,0.605090,0.385661,0.365713,0.390981
1,0.011488,0.455654,0.449910,0.445028,0.496837,0.380698
2,0.020263,0.439030,0.459293,0.432276,0.506574,0.390399
3,0.031189,0.436649,0.405460,0.405460,0.514622,0.463394
4,0.028571,0.490270,0.420558,0.418844,0.553125,0.317133


### Encoder para los numéricos

In [187]:
categoricos = X.select_dtypes(include='object')
ohe = OneHotEncoder(sparse_output=False) 
encoded_categoricos = ohe.fit_transform(categoricos)
encoded_df = pd.DataFrame(encoded_categoricos, columns=ohe.get_feature_names_out(categoricos.columns))
encoded_df.head()

,gender_F,gender_M,ssc_b_Central,ssc_b_Others,hsc_b_Central,hsc_b_Others,hsc_s_Arts,hsc_s_Commerce,hsc_s_Science,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,workex_No,workex_Yes,specialisation_Mkt&Fin,specialisation_Mkt&HR
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


### Juntar las caracteristicas

In [188]:
df_final = pd.concat((numericos_norm, encoded_df, y), axis=1)
df_final.head()
df_final.to_csv('data/clean_data/campus_selection_limpio.csv')

## 2.- Marketing Response

In [189]:
ruta = 'data/raw_data/marketing_response/train.csv'
df = pd.read_csv(ruta)
df.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


### Imbalance Ratio

In [190]:
maximo = df['y'].value_counts().max()
minimo = df['y'].value_counts().min()
IR = np.round(maximo/minimo,4)
print(f'IR: {IR}')

IR: 2.2443


### Mapeo valores etiqueta

In [191]:
df['y'].replace({'no':0, 'yes':1}, inplace=True)

/tmp/ipykernel_4241/3818101257.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['y'].replace({'no':0, 'yes':1}, inplace=True)
/tmp/ipykernel_4241/3818101257.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['y'].replace({'no':0, 'yes':1}, inplace=True)


### Repartición de características y clase

In [192]:
X = df.iloc[:,:-1]
y = df['y']
print(X.shape)
print(y.shape)

(12870, 16)
(12870,)


### Numéricos y categóricos

In [193]:
numericos = X.select_dtypes(include='number')
categoricos = X.select_dtypes(include='object')

### Encoder para los categóricos

In [194]:
ohe = OneHotEncoder(sparse_output=False)
encoded_categoricos = ohe.fit_transform(categoricos)
encoded_categoricos = pd.DataFrame(encoded_categoricos, columns=ohe.get_feature_names_out(categoricos.columns))
encoded_categoricos.shape

(12870, 44)

### Normalizar los numéricos

In [195]:
normalizer = Normalizer()
numericos_norm = normalizer.fit_transform(numericos)
numericos_norm = pd.DataFrame(numericos_norm, columns = numericos.columns)
numericos_norm.shape

(12870, 7)

### Unir DF final

In [196]:
df_final = pd.concat((numericos_norm, encoded_categoricos, y), axis=1)
df_final.head()

,ID,age,balance,day,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y
0,0.603863,0.001266,0.797087,0.000480,0.000087,-0.000044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.999745,0.001146,0.022572,0.000220,0.000132,-0.000044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.999913,0.002846,0.012806,0.001328,0.000190,-0.000095,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0.999993,0.002995,0.002192,0.000731,0.000073,-0.000073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.999594,0.002388,0.028386,0.000177,0.000088,-0.000088,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0


In [197]:
df_final.to_csv('data/clean_data/bank_churn_limpio.csv')

## 3.- Loan Aproval

In [198]:
ruta = 'data/raw_data/loan_aproval/loan_data.csv'
df = pd.read_csv(ruta)
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


### Imbalance ratio

In [200]:
mayor = df['loan_status'].value_counts().max()
menor = df['loan_status'].value_counts().min()
IR = mayor/menor
print(f'IR: {IR}')

IR: 3.5


### Separar caracteristicas y etiqueta

In [201]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

### Separar numpericos y categoricos

In [202]:
numericos = X.select_dtypes(include='number')
categoricos = X.select_dtypes(include='object')

### Encoder para categoricos

In [203]:
ohe = OneHotEncoder(sparse_output=False)
encoded_df = ohe.fit_transform(categoricos)
encoded_df = pd.DataFrame(encoded_df, columns= ohe.get_feature_names_out(categoricos.columns))

### Normalizar numéricos

In [204]:
normalizer = Normalizer()
numericos_norm = normalizer.fit_transform(numericos)
numericos_norm = pd.DataFrame(numericos_norm, columns=numericos.columns)

### Unir df final

In [205]:
df_final = pd.concat((numericos_norm, encoded_df, y), axis=1)
df_final.head()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,person_gender_female,person_gender_male,person_education_Associate,person_education_Bachelor,person_education_Doctorate,person_education_High School,person_education_Master,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,previous_loan_defaults_on_file_No,previous_loan_defaults_on_file_Yes,loan_status
0,0.000275,0.899221,0.000000,0.437437,0.000200,0.000006,0.000037,0.007011,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1
1,0.001703,0.995867,0.000000,0.081083,0.000903,0.000006,0.000162,0.040866,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.001836,0.913577,0.000220,0.403977,0.000945,0.000032,0.000220,0.046641,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
3,0.000264,0.915674,0.000000,0.401848,0.000175,0.000005,0.000023,0.007750,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
4,0.000321,0.883831,0.000013,0.467741,0.000191,0.000007,0.000053,0.007831,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1


### Guardar df limpio

In [206]:
df_final.to_csv('data/clean_data/loan_aproval_clean.csv')